In [1]:
from openai import OpenAI
import openai

In [2]:
%run utilities.ipynb

In [16]:
def prompt_model_over_api(modelname, messages):

    client = OpenAI(api_key = "EMPTY", base_url = "http://mlserver.iteratec.de:8000/v1")

    chat_completion = '' 
    try: 
        chat_completion = client.chat.completions.create(
            model = modelname,
            messages = messages
        )

    except openai.APIConnectionError as e:
        print("The server could not be reached")
        print(e.__cause__)  # an underlying Exception, likely raised within httpx.
    except openai.RateLimitError as e:
        print("A 429 status code was received; we should back off a bit.")
    except openai.APIStatusError as e:
        print("Another non-200-range status code was received")
        print(e.status_code)
        print(e.response)
    except openai.BadRequestError as e:
        print(e.response)
    except openai.InternalServerError as e:
        print(e.response)
    except openai.UnprocessableEntityError as e:
        print(e.reponse)
    except openai.NotFoundError as e:
        print(e.response)
    except openai.PermissionDeniedError as e:
        print(e.response)

    return chat_completion.choices[0].message.content

# prompt_model_over_api("casperhansen/mixtral-instruct-awq", [{"role": "user", "content": 'system_message'}])

In [43]:
def save_results():
    timestamp = str(int(time.time()))

    # create the file name
    if 'None' in prompt_dropdown.value:
        prompt_value = 'NoContext'
    else:
        prompt_value = 'SpecificContext'

    if minimal_target_value_input.value:
        target = f'min_target_strength_{minimal_target_value_input.value}'
    else:
        target = 'min_target_strength_not_specified'

    if model_dropdown.value == 'casperhansen/mixtral-instruct-awq':
        model_value = 'mixtral-instruct-awq'
    else:
        model_value = model_dropdown.value

    filename = f"../results/LLM/{model_value}_{prompt_value}_prompt_experiment_{experiment+1}_{target}_Dev_Cycles_{number_development_input.value}_test_time_{test_time_dropdown.value}_{timestamp}_{prompt_dropdown.value}.csv"
       
        # open the file in write mode
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        # write the headers
        writer.writerow(["Formulation", "Compressive Strength"])

        # iterate over the training data
        for data in training_data:
            try:
                # parse the data to extract formulation and compressive strength
                formulation, strength_str = data.split(" resulted in a strength of ")
                strength = float(strength_str.split(" ")[0])  # convert string to float
                writer.writerow([formulation, strength])
            except ValueError as e:
                print(f"Failed to parse data: {data}")
                print(f"Error: {e}")

        print(f"Data for experiment {experiment+1} successfully saved to {filename}. \n\n")

In [3]:
def prompt_model_with_extended_test_time(modelname, training_data, messages_inverse_design):
    predictions = []
    formulations = []
    unique_formulations = []
    retry_count = 0
    max_retries = 10

    # Add the new user prompt to ask for three unique formulations, if no previous conversation history, last message is from and just appending another user message causes api error
    # If last message from user, append additional message to content else append entirely new message
    if messages_inverse_design[-1]["role"] == "user":     
        messages_inverse_design[-1]["content"] += ("\n Output 3 three completly new and untested formulations with a extremly high expected compressive strength , each of the three must adhere to the given structure and parameter limits!.")
    else:
        messages_inverse_design.append({"role": "user", "content": "Output 3 three completly new and untested formulations with a extremly high expected compressive strength , each of the three must adhere to the given structure and parameter limits!."})
    


    while not predictions and retry_count < max_retries:        
        # Make API call
        model_response_inverse_design = prompt_model_over_api(model_dropdown.value, messages_inverse_design)
        # Extract the three formulations from the model response
        response_lines = model_response_inverse_design.split('\n')
        # Define a regular expression pattern to capture the relevant information
        pattern = re.compile(r'Powderkg\s*=\s*(\d+),\s*wc\s*=\s*(\d+\.\d+),\s*materials\s*=\s*(\d+\.\d+/\d+\.\d+),\s*curing\s*=\s*(\w+)', re.IGNORECASE)
        for line in response_lines:
            if line.strip():
                match = pattern.search(line)
                if match:
                    formulations.append(line)

        # formulations = [line for line in response_lines if line.startswith("The formulation is")]
        if training_data:
            training_formulations = extract_formulations_from_training_data(training_data)
            print(f'Current training formulations :\n {training_formulations}\n')
            unique_formulations = [f for f in formulations if f not in training_formulations]
            print(f'{len(unique_formulations)} unique formulations were found by model during inverse design...')
            if len(unique_formulations) == 0:
                print(f'no new formulations extracted, {formulations} already in training data')
        if not training_data:
            unique_formulations = formulations
        print(f'These are the three novel formulations: \n{unique_formulations}\n')
        
        # # Prepare the forward task prompt
        system_message = DA_role_prompt.value  + '\n' + context_prompt.value + '\n'
        if not training_data:
            forward_prompt_base = f"{VM_role_text}"
        else:
            forward_prompt_base = f"////Previous Formulation and Lab Validation:" + "\n".join(training_data) + f"\n{VM_role_text}"
            # print(f'This is forward prompt {forward_prompt_base}')


        # print(messages)
        for idx, formulation in enumerate(unique_formulations):
            # Create the full forward task prompt
            forward_prompt = system_message + f"{forward_prompt_base}\n Considering this context, what is the compressive strength of {formulation}? Answer in this exact format: {'your estimate'} MPa"
            messages_forward_prediction = [{"role": "user", "content": forward_prompt}]    

            model_response_forward_prediction = prompt_model_over_api(model_dropdown.value, messages_forward_prediction)

            
                # Extract the predicted strength from the model's response
            try:
                correct_output_format = re.search(r'(\d+(\.\d+)?)\s*MPa', model_response_forward_prediction)
                if correct_output_format:
                    predicted_strength_str = correct_output_format.group(1)  # This contains just the 'number' part
                    predicted_strength = float(predicted_strength_str)
                    if (predicted_strength < 25):
                        print(f'Unusual result for following model response : \n{model_response_forward_prediction}')
                        continue
                    print(f'Verifier Model forward strength prediction for {formulation} :\n{predicted_strength}\n')
                else:
                    print(f"No correct output format for MPa found in forward model prediction {idx}. \n {model_response_forward_prediction} \n")
                    continue
            except ValueError:
                print(f"Could not convert predicted strength for {formulation} to float. Skipping this formulation.")
                continue

            # Append to the predictions list
            predictions.append((formulation, predicted_strength))
        if predictions:
            break   
        if not predictions:
            retry_count += 1
            print(f"No valid predictions were made on attempt {retry_count}. Attempting again...")
    
    if not predictions:
        print("No valid predictions were made. Defaulting to the first formulation.")
        return {'role': 'assistant', 'content': formulations[0] if formulations else 'Unable to make valid predictions.'}
    # # Select the formulation with the highest predicted strength
    final_response, best_strength = max(predictions, key=lambda x: x[1])
    print(f'Formulation chosen: {final_response}')
    return final_response

In [4]:
for experiment in range(number_experiments_input.value):
    training_data = []
    tested_formulations = []
    print(f'Starting experiment number {experiment+1} ...\n')
    system_message =  DA_role_prompt.value  + '\n' + context_prompt.value
    highest_strength = 0.0

    for dev_cycle in range(number_development_input.value):
        print(f'Development cycle : {dev_cycle+1} \n')
        current_strength = 0.0
        messages = [{"role": "user", "content": system_message}]

        if training_data:
                messages.append({"role": "assistant", "content": "Previously, we have tested these formulations:\n" + "\n".join(tested_formulations)})
                messages.append({"role": "user", "content": iterate_prompt.value})


        valid_solution = False
        while not valid_solution:
            if test_time_dropdown.value == '3':
                print('Extending test time...')
                model_response = prompt_model_with_extended_test_time(model_dropdown.value, training_data, messages)
            else:
                model_response = prompt_model_over_api(model_dropdown.value, messages)
            # print("--- Conversation History ---")
            # for msg in messages:
            #     print(f"{msg['role']}: {msg['content']} \n")
    
            # print(f'Current training data : {training_data}')
            parsed_solution = parse_solution(model_response)

            if parsed_solution is not None:
                (lab_result, new_training_data_point) = find_matching_result(formulation_df, parsed_solution)
                            
                if lab_result:
                    current_strength = lab_result
                    if current_strength > highest_strength:
                        highest_strength = current_strength
                    print(f'Lab data discovered for the following formulation : {new_training_data_point}')
                    training_data.append(f"{new_training_data_point} resulted in a strength of {current_strength} MPa.")
                    tested_formulations.append(f"{new_training_data_point}")
                                      
                    valid_solution = True
                                
                    print(f'The suggested formulation achieved a strength of {current_strength} MPa. \n\n')
                else:
                    print(f"Development cycle {dev_cycle+1}: No matching lab result found for suggestion {parsed_solution} \n\n")
            if not valid_solution:
                print(f"Development cycle {dev_cycle+1}: Model's response did not contain a valid solution. Trying again. \n\n")
                messages[-1]["content"] = iterate_prompt.value + "\nRemember the exact parameter grid: Powderkg: {360, 370, 380, 390,400, 410, 420, 430, 440, 450}, wc: {0.45, 0.5, 0.55, 0.6}, materials: {0.7/0.3, 0.6/0.4, 0.5/0.5}, curing: {ambient, heat}. It is extremly important that you stick to these values and reply in the following format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'"
       
        if minimal_target_value_input.value and current_strength >= minimal_target_value_input.value:
            print(f"\nDesired compressive strength of {minimal_target_value_input.value} MPa achieved after {dev_cycle+1} iterations. The solution is {parsed_solution}. \n\n")
            break
        
    print(f'We have tested the following formulations : {tested_formulations} \n\n')
    print(f'The highest strength out of the formulation we predicted is : {highest_strength} \n\n')

    save_results()





NameError: name 'number_experiments_input' is not defined